<a href="https://colab.research.google.com/github/myxa/coursework_2020/blob/master/%D0%BA%D1%83%D1%80%D1%81%D0%BE%D0%B2%D0%B0%D1%8F.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Разговорный чат-бот

In [1]:
import pandas as pd
import numpy as np

Обучающий датасет состоит из личных сообщений из вк за почти 10 лет. Также были добавлены диалоги из фильмов.

In [6]:
data = pd.read_csv('/content/drive/My Drive/колаб/курсовая/merged.csv')

In [9]:
data.sample(10)

,context,reply
63416,это правда ?,нет .
18352,А шо ты вдруг вспомнил,та мне для анкеты нада указать када я учился в...
8533,нарисовал чо-нить ? ? ?,Та
72253,а зачем мне перчатки ?,чтобы проголосовать .
36244,Из дк это,а
80427,это было бы плохо .,он бы уже был великим рыцарем .
115545,и он исчезнет .,и снова вернется .
36588,поооч,Гуашь_(((((
33590,ну да ладно,Ну да ладно
119044,"всё совершенно нормально "" . так , что йен нич...",почему бы тебе не заглядывать к нему время от ...


$X$ -- context (чье-то сообщение), таргет -- reply (мой ответ). В процессе отчистки сообщений от служебных символов пары контекст-ответ могли поменяться местами, но это не критично, я думаю.

# TF-IDF

Попытка создать не нейросетевой алгоритм для решения этой задачи

In [12]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD

from sklearn.neighbors import BallTree
from sklearn.base import BaseEstimator

Сначала получаем представления предложений как TF-IDF векторы  

In [14]:
tf = TfidfVectorizer()
bag = tf.fit(data['context'])
transformed = bag.transform(data['context'])

Далее используем SVD-разложение для уменьшения размерности

In [15]:
svd = TruncatedSVD(n_components=500)
small = svd.fit(transformed)

vecs = small.transform(transformed)

Сам алгоритм устроен просто -- кластеризуем все векторы context и потом просто берем рандомный вектор из пяти ближайших и используем его как ответ модели. 

In [17]:
def softmax(x):
  return np.exp(x)/sum(np.exp(x))

class NeighborSampler(BaseEstimator):
  def __init__(self, k=5, temp=1.0):
    self.k = k
    self.temp = temp
  def fit(self, X, y):
    self.tree_ = BallTree(X)
    self.y_ = np.array(y)
    
  def predict(self, X, random_state=42):
    dist, ind = self.tree_.query(X, return_distance=True, k=self.k)

    result = []
    for d, i in zip(dist, ind):
      p=softmax(d * self.temp)
      result.append(np.random.choice(i))

    return self.y_[result]

### Примеры

Обучаем модель на ответах

In [22]:
from sklearn.pipeline import make_pipeline
sampler = NeighborSampler(k=5, temp=0.7)
sampler.fit(vecs, data['reply'])
pipe = make_pipeline(tf, svd, sampler)

In [35]:
def inference():
  """ просто говорилка """
  while True:
    inp = input()
    answer = pipe.predict([inp])[0]
    print(answer)


Некоторые примеры, которые показались мне до смешного точными

In [38]:
pipe.predict(['приветик'])[0]

'здравствуй - здравствуй .'

In [ ]:
pipe.predict(['что же делать?'])

array(['для начала сесть и начать уже говорить чертову правду .'],
      dtype=object)

In [39]:
pipe.predict(['как дела?'])

array(['все хорошо .'], dtype=object)

In [ ]:
pipe.predict(['пошли покушаем'])

array(['быстрее !'], dtype=object)

In [ ]:
pipe.predict(['я спать'])

array(['ну спать так спать'], dtype=object)

In [ ]:
pipe.predict(['ты где?'])

array(['На адмиралтейской'], dtype=object)

In [ ]:
pipe.predict(['андрюха'])

array(['А шо нитак'], dtype=object)

In [ ]:
pipe.predict(['ничо особенного'])

array(['АВЗХАВЗАЗВЗХХ'], dtype=object)

In [ ]:
pipe.predict(['ты Ленин'])

array(['ДААААААА'], dtype=object)

# seq2seq

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np

import spacy

import random
import math
import time
 
import torch.nn.functional as F
from torchtext.data import Field, BucketIterator, TabularDataset

## Предобработка данных

In [ ]:
seed = 43

random.seed(seed)
torch.manual_seed(seed)
np.random.seed(seed)
torch.backends.cudnn.deterministic = True

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

movie = '/content/drive/My Drive/курсовая/movie.csv'
merged = '/content/drive/My Drive/курсовая/merged.csv'
my = '/content/drive/My Drive/курсовая/my.csv'

### messages

Здесь было решено разделить датасеты, потому что оба два за раз colab не вывозил

In [ ]:
context = Field(tokenize = 'spacy',
            init_token = '<sos>', 
            eos_token = '<eos>', 
            lower = True)

reply = Field(tokenize = 'spacy',
            init_token = '<sos>', 
            eos_token = '<eos>', 
            lower = True)

In [ ]:
dataset = TabularDataset(my, 'csv', 
                          [('context', context), ("reply", reply)], 
                          skip_header=True)

train, test = dataset.split(0.7)

In [ ]:
batch_size = 16
train_iterator, valid_iterator = BucketIterator.splits(
    (train, test), 
     batch_size = batch_size,
     sort_within_batch = True,
     sort_key = lambda x : len(x.context),
     device = device)

In [ ]:
print('train len:', len(train.examples))
print('test len:', len(test.examples), '\n')

context.build_vocab(train, min_freq = 2)
reply.build_vocab(train, min_freq = 2)
print("Number of words in context", len(context.vocab))
print("Number of words in reply", len(reply.vocab),'\n')

for i in range(10):
  print(train[i].context, train[i].reply)

train len: 43716
test len: 18736 

Number of words in context 9334
Number of words in reply 8456 

['жиза'] ['ввс', 'перестанет', 'сотрудничать', 'с', 'первым', ',', 'вот', 'так', 'вот']
['чо', 'это'] ['чо', 'фасоль', 'фу']
['реву'] ['какую', 'книгу']
['я', 'удалил', '6', 'гб', 'ненужного', 'хлама', '.'] ['продолжай', 'в', 'том', 'же', 'духе']
['латна', '.'] ['.']
['што'] ['inj']
['то', 'есть', 'седня', 'ты', 'халявничаешь'] ['я', 'читаю']
['это', 'што'] ['раксакорикофаллапаториус']
['а', 'ты', 'чо', 'тош', 'в', 'школе'] ['тааа']
['что', 'делаешь', '?'] ['та', 'чот', 'сижу']


### movie dialogs

In [ ]:
context = Field(tokenize = 'spacy',
            init_token = '<sos>', 
            eos_token = '<eos>', 
            lower = True)

reply = Field(tokenize = 'spacy',
            init_token = '<sos>', 
            eos_token = '<eos>', 
            lower = True)

In [ ]:
dataset = TabularDataset(movie, 'csv', 
                          [('context', context), ("reply", reply)], 
                          skip_header=True)

train, test = dataset.split(0.7)

In [ ]:
batch_size = 16
train_iterator, valid_iterator = BucketIterator.splits(
    (train, test), 
     batch_size = batch_size,
     sort_within_batch = True,
     sort_key = lambda x : len(x.context),
     device = device)

In [ ]:
print('train len:', len(train.examples))
print('test len:', len(test.examples), '\n')

context.build_vocab(train, min_freq = 5)
reply.build_vocab(train, min_freq = 5)
print("Number of words in context", len(context.vocab))
print("Number of words in reply", len(reply.vocab),'\n')

for i in range(10):
  print(train[i].context, train[i].reply)

train len: 42034
test len: 18015 

Number of words in context 4497
Number of words in reply 3443 

['я', 'пойму', ',', 'если', 'ты', 'не', 'сообщишь', 'семье', 'прямо', 'сейчас', '.'] ['как', 'ты', 'не', 'поймёшь', ',', 'мы', 'друзья', '.']
['правда', '?'] ['конечно', 'правда', '!']
['на', 'днище', 'ее', 'машины', 'я', 'нашел', 'gps', '-', 'датчик', 'слежения', '.'] ['по', 'данным', 'твоего', 'gps', 'ты', 'в', 'трех', 'с', 'половиной', 'милях', 'от', 'него', '.']
['не', 'убивайте', 'меня', ',', 'пожалуйста', '.'] ['пожалуйста', 'ответь', 'на', 'вопрос', '.']
['я', 'знаю', 'кое', '-', 'кого', 'кто', 'тебя', 'подвезет', '.'] ['кого', '?']
['нравится', '?'] ['да', ',', 'тебе', 'идёт', '.']
['да', ',', 'дождь', '.'] ['не', 'хочу', ',', 'чтобы', 'ты', 'шёл', '.']
['ну', ',', 'когда', 'у', 'него', 'есть', 'работа', '.', 'что', 'не', 'так', 'часто', 'в', 'последнее', 'время', '.'] ['я', 'потерял', 'свою', 'работу', '.', '.', '.', '.', 'учитывая', 'некоторые', 'странные', 'обстоятельства', '.'

##Seq2seq модель с attention

Получилось очень плохо! 

In [ ]:
class Encoder(nn.Module):
    def __init__(self, input_dim, emb_dim, enc_hid_dim, dec_hid_dim, dropout):
        super().__init__()
        
        self.input_dim = input_dim
        self.emb_dim = emb_dim
        self.enc_hid_dim = enc_hid_dim
        self.dec_hid_dim = dec_hid_dim
        self.dropout = dropout
        
        self.embedding = nn.Embedding(input_dim, emb_dim)
        
        self.rnn = nn.GRU(emb_dim, enc_hid_dim, bias=False)
        
        self.fc = nn.Linear(enc_hid_dim, dec_hid_dim, bias=False)
        
        self.dropout = nn.Dropout(dropout)
        

    def forward(self, src, src_len):
        
        #src = [src sent len, batch size]
        #src_len = [src sent len]
        
        embedded = self.dropout(self.embedding(src))
        
        #embedded = [src sent len, batch size, emb dim]
        
        packed_embedded = nn.utils.rnn.pack_padded_sequence(embedded, src_len)
        
        packed_outputs, hidden = self.rnn(packed_embedded)
                     
        #packed_outputs is a packed sequence containing all hidden states
        #hidden is now from the final non-padded element in the batch
            
        outputs, _ = nn.utils.rnn.pad_packed_sequence(packed_outputs) 
            
        #outputs is now a non-packed sequence, all hidden states obtained
        #  when the input is a pad token are all zeros
            
        #outputs = [sent len, batch size, hid dim * num directions]
        #hidden = [n layers * num directions, batch size, hid dim]
        
        #hidden is stacked [forward_1, backward_1, forward_2, backward_2, ...]
        #outputs are always from the last layer
        
        #hidden [-2, :, : ] is the last of the forwards RNN 
        #hidden [-1, :, : ] is the last of the backwards RNN
        
        #initial decoder hidden is final hidden state of the forwards and backwards 
        #  encoder RNNs fed through a linear layer
        hidden = torch.tanh(self.fc(hidden[-1,:,:]))
        
        #outputs = [sent len, batch size, enc hid dim * 2]
        #hidden = [batch size, dec hid dim]
        
        return outputs, hidden

In [ ]:
class Attention(nn.Module):
    def __init__(self, enc_hid_dim, dec_hid_dim):
        super().__init__()
        
        self.enc_hid_dim = enc_hid_dim
        self.dec_hid_dim = dec_hid_dim
        
        self.attn = nn.Linear(enc_hid_dim + dec_hid_dim, dec_hid_dim, bias=False)
        self.v = nn.Parameter(torch.rand(dec_hid_dim))
        
    def forward(self, hidden, encoder_outputs):
        
        #hidden = [batch size, dec hid dim]
        #encoder_outputs = [src sent len, batch size, enc hid dim * 2]
        #mask = [batch size, src sent len]
        
        batch_size = encoder_outputs.shape[1]
        src_len = encoder_outputs.shape[0]
        
        #repeat encoder hidden state src_len times
        hidden = hidden.unsqueeze(1).repeat(1, src_len, 1)
        
        encoder_outputs = encoder_outputs.permute(1, 0, 2)
        
        #hidden = [batch size, src sent len, dec hid dim]
        #encoder_outputs = [batch size, src sent len, enc hid dim * 2]
        
        energy = torch.tanh(self.attn(torch.cat((hidden, encoder_outputs), dim = 2))) 
        
        #energy = [batch size, src sent len, dec hid dim]
                
        energy = energy.permute(0, 2, 1)
        
        #energy = [batch size, dec hid dim, src sent len]
        
        #v = [dec hid dim]
        
        v = self.v.repeat(batch_size, 1).unsqueeze(1)
        
        #v = [batch size, 1, dec hid dim]
            
        attention = torch.bmm(v, energy).squeeze(1)
        
        #attention = [batch size, src sent len]
      
        
        return F.softmax(attention, dim = 1)

In [ ]:
class Decoder(nn.Module):
    def __init__(self, output_dim, emb_dim, enc_hid_dim, dec_hid_dim, dropout, attention):
        super().__init__()

        self.emb_dim = emb_dim
        self.enc_hid_dim = enc_hid_dim
        self.dec_hid_dim = dec_hid_dim
        self.output_dim = output_dim
        self.dropout = dropout
        self.attention = attention
        
        self.embedding = nn.Embedding(output_dim, emb_dim)
        
        self.rnn = nn.GRU(enc_hid_dim + emb_dim, dec_hid_dim)
        
        self.out = nn.Linear(enc_hid_dim + dec_hid_dim + emb_dim, output_dim, bias=False)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, input, hidden, encoder_outputs):
             
        #input = [batch size]
        #hidden = [batch size, dec hid dim]
        #encoder_outputs = [src sent len, batch size, enc hid dim * 2]
        #mask = [batch size, src sent len]
        
        input = input.unsqueeze(0)
        
        #input = [1, batch size]
        
        embedded = self.dropout(self.embedding(input))
        
        #embedded = [1, batch size, emb dim]
        
        a = self.attention(hidden, encoder_outputs)
                
        #a = [batch size, src sent len]
        
        a = a.unsqueeze(1)
        
        #a = [batch size, 1, src sent len]
        
        encoder_outputs = encoder_outputs.permute(1, 0, 2)
        
        #encoder_outputs = [batch size, src sent len, enc hid dim * 2]
        
        weighted = torch.bmm(a, encoder_outputs)
        
        #weighted = [batch size, 1, enc hid dim * 2]
        
        weighted = weighted.permute(1, 0, 2)
        
        #weighted = [1, batch size, enc hid dim * 2]
        
        rnn_input = torch.cat((embedded, weighted), dim = 2)
        
        #rnn_input = [1, batch size, (enc hid dim * 2) + emb dim]
            
        output, hidden = self.rnn(rnn_input, hidden.unsqueeze(0))
        
        #output = [sent len, batch size, dec hid dim * n directions]
        #hidden = [n layers * n directions, batch size, dec hid dim]
        
        #sent len, n layers and n directions will always be 1 in this decoder, therefore:
        #output = [1, batch size, dec hid dim]
        #hidden = [1, batch size, dec hid dim]
        #this also means that output == hidden
        assert (output == hidden).all()
        
        embedded = embedded.squeeze(0)
        output = output.squeeze(0)
        weighted = weighted.squeeze(0)
        
        output = self.out(torch.cat((output, weighted, embedded), dim = 1))
        
        #output = [bsz, output dim]
        
        return output, hidden.squeeze(0), a.squeeze(1)

In [ ]:
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, pad_idx, sos_idx, eos_idx, device):
        super().__init__()
        
        self.encoder = encoder
        self.decoder = decoder
        self.pad_idx = pad_idx
        self.sos_idx = sos_idx
        self.eos_idx = eos_idx
        self.device = device

        
    def forward(self, src, src_len, trg, teacher_forcing_ratio = 0.5):
        
        #src = [src sent len, batch size]
        #src_len = [batch size]
        #trg = [trg sent len, batch size]
        #teacher_forcing_ratio is probability to use teacher forcing
        #e.g. if teacher_forcing_ratio is 0.75 we use teacher forcing 75% of the time
        
        if trg is None:
            assert teacher_forcing_ratio == 0, "Must be zero during inference"
            inference = True
            trg = torch.zeros((100, src.shape[1])).long().fill_(self.sos_idx).to(src.device)
        else:
            inference = False
            
        batch_size = src.shape[1]
        max_len = trg.shape[0]
        trg_vocab_size = self.decoder.output_dim
        
        #tensor to store decoder outputs
        outputs = torch.zeros(max_len, batch_size, trg_vocab_size).to(self.device)
        
        #tensor to store attention
        attentions = torch.zeros(max_len, batch_size, src.shape[0]).to(self.device)
        
        #encoder_outputs is all hidden states of the input sequence, back and forwards
        #hidden is the final forward and backward hidden states, passed through a linear layer
        encoder_outputs, hidden = self.encoder(src, src_len)
                
        #first input to the decoder is the <sos> tokens
        output = trg[0,:]
        
                
        for t in range(1, max_len):
            output, hidden, attention = self.decoder(output, hidden, encoder_outputs)
            outputs[t] = output
            attentions[t] = attention
            teacher_force = random.random() < teacher_forcing_ratio
            top1 = output.max(1)[1]
            output = (trg[t] if teacher_force else top1)
            if inference and output.item() == self.eos_idx:
                return outputs[:t], attentions[:t]
            
        return outputs, attentions

In [ ]:
INPUT_DIM = len(context.vocab)
OUTPUT_DIM = len(reply.vocab)
ENC_EMB_DIM = 150
DEC_EMB_DIM = 150
ENC_HID_DIM = 100
DEC_HID_DIM = 100
ENC_DROPOUT = 0.7
DEC_DROPOUT = 0.7
PAD_IDX = context.vocab.stoi['<pad>']
SOS_IDX = reply.vocab.stoi['<sos>']
EOS_IDX = reply.vocab.stoi['<eos>']

attn = Attention(ENC_HID_DIM, DEC_HID_DIM)
enc = Encoder(INPUT_DIM, ENC_EMB_DIM, ENC_HID_DIM, DEC_HID_DIM, ENC_DROPOUT)
dec = Decoder(OUTPUT_DIM, DEC_EMB_DIM, ENC_HID_DIM, DEC_HID_DIM, DEC_DROPOUT, attn)

model = Seq2Seq(enc, dec, PAD_IDX, SOS_IDX, EOS_IDX, device).to(device)


In [ ]:
def init_weights(m):
    for name, param in m.named_parameters():
        if 'weight' in name:
            nn.init.normal_(param.data, mean=0, std=0.01)
        else:
            nn.init.constant_(param.data, 0)
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'Модель содержит {count_parameters(model):,} параметров')            
model.apply(init_weights)

Модель содержит 2,606,750 параметров


Seq2Seq(
  (encoder): Encoder(
    (embedding): Embedding(4497, 150)
    (rnn): GRU(150, 100, bias=False)
    (fc): Linear(in_features=100, out_features=100, bias=False)
    (dropout): Dropout(p=0.7, inplace=False)
  )
  (decoder): Decoder(
    (attention): Attention(
      (attn): Linear(in_features=200, out_features=100, bias=False)
    )
    (embedding): Embedding(3443, 150)
    (rnn): GRU(250, 100)
    (out): Linear(in_features=350, out_features=3443, bias=False)
    (dropout): Dropout(p=0.7, inplace=False)
  )
)

In [ ]:
def train(model, iterator, optimizer, criterion, clip):
    
    model.train()
    
    epoch_loss = 0
    
    for i, batch in enumerate(iterator):
        
        #src, src_len = batch.src
        src = batch.context
        src_len_np = np.full((src.shape[1]), src.shape[0])
        src_len = torch.tensor(src_len_np)
        trg = batch.reply
        
        optimizer.zero_grad()
        
        output, attetion = model(src, src_len, trg, 0.2)
        
        #trg = [trg sent len, batch size]
        #output = [trg sent len, batch size, output dim]
        
        output = output[1:].view(-1, output.shape[-1])
        trg = trg[1:].view(-1)
        
        #trg = [(trg sent len - 1) * batch size]
        #output = [(trg sent len - 1) * batch size, output dim]
        
        loss = criterion(output, trg)
        
        loss.backward()
        
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        
        optimizer.step()
        
        epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

In [ ]:
def evaluate(model, iterator, criterion):
    
    model.eval()
    
    epoch_loss = 0
    
    with torch.no_grad():
    
        for i, batch in enumerate(iterator):

            #src, src_len = batch.src
            src = batch.context
            src_len_np = np.full((src.shape[1]), src.shape[0])
            src_len = torch.tensor(src_len_np)
            trg = batch.reply

            output, attention = model(src, src_len, trg, 0) #turn off teacher forcing

            #trg = [trg sent len, batch size]
            #output = [trg sent len, batch size, output dim]

            output = output[1:].view(-1, output.shape[-1])
            trg = trg[1:].view(-1)

            #trg = [(trg sent len - 1) * batch size]
            #output = [(trg sent len - 1) * batch size, output dim]

            loss = criterion(output, trg)

            epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

In [ ]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [ ]:
optimizer = torch.optim.Adam(model.parameters(), 0.001)
criterion = nn.CrossEntropyLoss(ignore_index = PAD_IDX)

N_EPOCHS = 10
CLIP = 1

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):
    
    start_time = time.time()
    
    train_loss = train(model, train_iterator, optimizer, criterion, CLIP)
    valid_loss = evaluate(model, valid_iterator, criterion)
    
    end_time = time.time()
    
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), '/content/drive/My Drive/state dicts/attention0002.pt')
    
    print(f'Эпоха: {epoch+1:02} | Время: {epoch_mins}m {epoch_secs}s')
    print(f'Перплексия (обучение): {math.exp(train_loss):7.3f}')
    print(f'Перплексия (валидация): {math.exp(valid_loss):7.3f}')

Эпоха: 01 | Время: 4m 19s
Перплексия (обучение):  92.833
Перплексия (валидация):  74.676
Эпоха: 02 | Время: 4m 20s
Перплексия (обучение):  71.441
Перплексия (валидация):  67.576
Эпоха: 03 | Время: 4m 18s
Перплексия (обучение):  63.109
Перплексия (валидация):  63.903
Эпоха: 04 | Время: 4m 22s
Перплексия (обучение):  57.419
Перплексия (валидация):  61.712
Эпоха: 05 | Время: 4m 19s
Перплексия (обучение):  53.085
Перплексия (валидация):  61.003
Эпоха: 06 | Время: 4m 19s
Перплексия (обучение):  49.626
Перплексия (валидация):  60.608
Эпоха: 07 | Время: 4m 21s
Перплексия (обучение):  47.006
Перплексия (валидация):  60.765
Эпоха: 08 | Время: 4m 21s
Перплексия (обучение):  44.795
Перплексия (валидация):  60.608
Эпоха: 09 | Время: 4m 23s
Перплексия (обучение):  42.797
Перплексия (валидация):  61.716


KeyboardInterrupt: ignored

In [ ]:
model.load_state_dict(torch.load('/content/drive/My Drive/state dicts/attention0002.pt'))

<All keys matched successfully>

In [ ]:
def answer(model, sentence):
    model.eval()
    tokenized = ['<sos>'] + [t.lower() for t in sentence.split(' ')] + ['<eos>']
    numericalized = [context.vocab.stoi[t] for t in tokenized] 
    sentence_length = torch.LongTensor([len(numericalized)]).to(device) 
    tensor = torch.LongTensor(numericalized).unsqueeze(1).to(device) 

    logits, attention = model(tensor, sentence_length, None, 0) 
    translation_tensor = torch.argmax(logits.squeeze(1), 1)
    decoded = [reply.vocab.itos[t] for t in translation_tensor]
    decoded, attention = translation[1:], attention[1:]
    return decoded, attention

In [ ]:
a, _ = answer(model,"что делаешь?")
print(" ".join(a))

я ,
